In [1]:
# Imports for API to get data

import csv
import requests
import pandas as pd
import numpy as np
import re
from lxml import html
import json

In [2]:
# Data Processing Imports

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
# ML Imports

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout, Lambda, Flatten
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from tensorflow.keras.utils import Sequence
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [4]:
# Countries to Model on
cols = ['date', 'China', 'Italy', 'South Korea', 'France', 'Spain', 'Germany', 'United States', 'United Kingdom', 'India']

# Fetch Data
url = 'https://covid.ourworldindata.org/data/ecdc/total_cases.csv'
response = requests.get(url).content.decode('utf-8')
cr = csv.reader(response.splitlines(), delimiter=',')

In [5]:
# Preprocessing Data to start the time series from the day 
# when cases hit 50 for each country. Further documentation available
# on request.
# Author: Archit Checker <archit.checker@alumni.ashoka.edu.in>

df = pd.DataFrame(cr, index=None,)
headers = df.iloc[0]
df  = pd.DataFrame(df.values[1:], columns=headers)
df = df[cols]
max_day = 0
def parse(val):
    val = re.sub('\D', '', val)
    return int(val)

current_X = []
for col in cols[1:]:
    day = 1
    arr = df[col]
    for element in list(arr):
        try:
            if int(element) >= 50:
                if col =="China" and int(element) == 59 and day == 2:
                    continue
                current_X.append([col, day, int(element)])
                day += 1
        except:
            if day!=1:
                day += 1
#   Special treatment for india - fetch latest datapoint from mohw.
    if col == "India":
        page = requests.get("https://www.mohfw.gov.in/")
        tree = html.fromstring(page.content)
        cases = []
        cases.append(tree.xpath('//*[@id="site-dashboard"]/div/div/div/div/ul/li[1]/strong/text()')[0])
        cases.append(tree.xpath('//*[@id="site-dashboard"]/div/div/div/div/ul/li[2]/strong/text()')[0])
        cases.append(tree.xpath('//*[@id="site-dashboard"]/div/div/div/div/ul/li[3]/strong/text()')[0])
        cases.append(tree.xpath('//*[@id="site-dashboard"]/div/div/div/div/ul/li[4]/strong/text()')[0])
        x = 0
        print(cases)
        for case in cases:
            x += parse(case)
        current_X[len(current_X)-1] = [col, day-1, x]
    if day > max_day:
        max_day = day
new_df = pd.DataFrame(current_X, columns=["country", "days", "cases"])
print(max_day)

['120406', '119292', '6929', '1']
145


In [6]:
# Grouping Data by Country

grouped = new_df.pivot(index = "days", columns = "country")
grouped = grouped.swaplevel(i = 0, j = 1, axis = 1)
df = grouped.reindex(sorted(grouped.columns), axis=1)

In [7]:
df

country,China,France,Germany,India,Italy,South Korea,Spain,United Kingdom,United States
,cases,cases,cases,cases,cases,cases,cases,cases,cases
days,,,,,,,,,
1,59.0,57.0,57.0,50.0,79.0,80.0,54.0,51.0,53.0
2,63.0,100.0,111.0,73.0,132.0,155.0,82.0,85.0,53.0
3,80.0,130.0,129.0,75.0,229.0,345.0,136.0,115.0,59.0
4,216.0,178.0,157.0,83.0,322.0,601.0,192.0,163.0,60.0
5,235.0,212.0,196.0,90.0,400.0,762.0,267.0,206.0,66.0
...,...,...,...,...,...,...,...,...,...
140,84159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,84160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Shifting time series to account for lengthy China History

i = 0
for country, param in df:
    try:
        norm_shift = np.where(df[country][param].isna())[0][0]
        df[country][param] = df[country][param].shift((max_day - norm_shift - 1))
    except:
        pass

for country, param in df:
        df[country][param] = df[country][param].fillna(0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [9]:
df

country,China,France,Germany,India,Italy,South Korea,Spain,United Kingdom,United States
,cases,cases,cases,cases,cases,cases,cases,cases,cases
days,,,,,,,,,
1,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,216.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
140,84159.0,151325.0,182370.0,207615.0,233515.0,11590.0,239932.0,277985.0,1831821.0
141,84160.0,151677.0,182764.0,216919.0,233836.0,11629.0,240326.0,279856.0,1851520.0


In [10]:
cols = list(df)
cols

[('China', 'cases'),
 ('France', 'cases'),
 ('Germany', 'cases'),
 ('India', 'cases'),
 ('Italy', 'cases'),
 ('South Korea', 'cases'),
 ('Spain', 'cases'),
 ('United Kingdom', 'cases'),
 ('United States', 'cases')]

In [11]:
# Basic rearrangement, moving india to last column

col = df.pop(('India', 'cases'))
df.insert(len(cols)-1, col.name, col)

In [12]:
df

country,China,France,Germany,Italy,South Korea,Spain,United Kingdom,United States,India
,cases,cases,cases,cases,cases,cases,cases,cases,cases
days,,,,,,,,,
1,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,216.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
140,84159.0,151325.0,182370.0,233515.0,11590.0,239932.0,277985.0,1831821.0,207615.0
141,84160.0,151677.0,182764.0,233836.0,11629.0,240326.0,279856.0,1851520.0,216919.0


In [13]:
# Dropping the superfluos column.
# Comment this out for more variables per country.

df = df.droplevel(1, axis = 1)

In [14]:
df

country,China,France,Germany,Italy,South Korea,Spain,United Kingdom,United States,India
days,,,,,,,,,
1,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,216.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
140,84159.0,151325.0,182370.0,233515.0,11590.0,239932.0,277985.0,1831821.0,207615.0
141,84160.0,151677.0,182764.0,233836.0,11629.0,240326.0,279856.0,1851520.0,216919.0
142,84171.0,152444.0,183271.0,234013.0,11668.0,240660.0,281661.0,1872660.0,226770.0


In [15]:
num_rows = len(df.values)
num_rows

144

In [16]:
# If modelling solely on China, uncomment this.

# df = df.drop(["France", "Germany", "Italy", "South Korea", "Spain", "United Kingdom", "United States"], axis=1)

In [17]:
# Preparation for scaling the data

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [18]:
df_val = df.values

In [19]:
# LSTM reshaping for input data.
# Code borrowed from: https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            print(i)
            print(end_ix)
            print(out_end_ix)
            print(len(sequences))
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [20]:
# Number of prior days to use for prediction, number of days to predict

n_steps_in, n_steps_out = 3, 5

In [21]:
X_samples = len(df_val) - n_steps_in - n_steps_out + 2
print(X_samples)

138


In [22]:
train_split = 0.85

In [23]:
num_train = int(train_split * X_samples)
num_test = X_samples - num_train

In [24]:
print(num_train)
print(num_test)

117
21


In [25]:
# Scaling X data

scaler_X.fit(df_val[:, :-1])

df_val[:, :-1] = scaler_X.transform(df_val[:, :-1])

In [26]:
# Scaling y data

scaler_y.fit(df_val[:, -1].reshape(-1, 1))

df_val[:, -1] = scaler_y.transform(df_val[:, -1].reshape(-1, 1)).reshape(num_rows)

In [27]:
# Train Data

train_X, train_y = split_sequences(df_val[:num_train], n_steps_in, n_steps_out)

111
114
118
117


In [28]:
# Test Data

test_X, test_y = split_sequences(df_val[num_train:], n_steps_in, n_steps_out)

21
24
28
27


In [29]:
train_X.shape, train_y.shape

((111, 3, 8), (111, 5))

In [30]:
n_features = train_X.shape[2]
print(n_features)

8


In [31]:
# Model architecture

model = Sequential()

model.add(LSTM(300, input_shape=(train_X.shape[1], n_features), return_sequences = True))
model.add(LSTM(150, recurrent_dropout=0.2, return_sequences = True))
model.add(LSTM(100, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(n_steps_out))
model.compile(loss='mae', optimizer=Adam(lr = .0001))

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 300)            370800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 150)            270600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 505       
Total params: 742,305
Trainable params: 742,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training loss

history = model.fit(train_X, train_y, epochs = 1500)
plt.plot(history.history['loss'], label='Train Loss')
plt.legend()
plt.show()

Train on 111 samples
Epoch 1/1500


In [ ]:
# Testing loss

model.evaluate(test_X, test_y)

In [ ]:
# Values predicted by the model

y_hat = model.predict(test_X[-1:])
y_hat = scaler_y.inverse_transform(y_hat)
predicted = y_hat.tolist()[0]
predicted

In [ ]:
# Actual values

actual_y = df_val[-5:, -1].reshape(-1, 1)
actual_y = scaler_y.inverse_transform(actual_y)
actuals = actual_y.tolist()
actuals = [x[0] for x in actuals]
actuals

In [ ]:
# Multiplication factor

factors = list(map(lambda x, y: x/y, actuals, predicted))
factors

In [ ]:
# Input to predict for next 5 days

x_new = df_val[-3:, :-1].reshape(1, 3, 8)

In [ ]:
# Prediction for next 5 days

y_new_hat = model.predict(x_new)
y_new_hat = scaler_y.inverse_transform(y_new_hat)
y_new_hat

In [ ]:
# Final (Scaled) predictions

final_predictions = y_new_hat.tolist()[0]
final_predictions = list(map(lambda x, y: x * y, y_new_hat, factors))
final_predictions